### Use the Frozen Lake environment.

https://www.gymlibrary.dev/environments/toy_text/frozen_lake/
#### **Importing required libraries**

In [1]:
import sys
import gym
import numpy as np

#### Intializing the environment

In [2]:
env = gym.make('FrozenLake-v1',is_slippery=True)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.9/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.9/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


#### Defining a **Helper function to calculate a state-value**

In [3]:
# Calculate a state-value function
def one_step_lookahead(env, state, V, discount):
    '''
    V: 2-D tensor
        Matrix of size nSxnA, each cell represents 
        a probability of taking actions
    '''
    
    n_actions = env.action_space.n
    action_values = np.zeros(shape=n_actions)
    for action in range(n_actions):
        for prob, next_state, reward, done in env.env.P[state][action]:
            action_values[action] += prob * (reward + discount * V[next_state])
    ''' 
    return:
        A vector of length nA containing the expected value of each action
    '''
    return action_values



#### Defining a function to **implement an optimal policy** for the Frozen Lake environment.

In [4]:
def policy_eval(policy, env, discount=1.0, theta=1e-9, max_iter=1000):
    """    
    policy: 2-D tensor
        Matrix of size nSxnA, each cell represents 
        a probability of taking actions
    """
    n_states = env.observation_space.n
    # number of evaluation iterations
    eval_iters = 1
    # initialize a value function for each state as zeros
    V = np.zeros(shape=n_states)
    # repeat until value change is below the threshold
    for i in range(int(max_iter)):
        # init a change of value function as zero
        delta = 0
        # iterate through each state
        for state in range(n_states):
            # init a new value of current state
            v = 0
            # Try all possible actions which can be taken from this state
            for action, action_prob in enumerate(policy[state]):
                # evaluate how good each next state will be
                for state_prob, next_state, reward, done in env.P[state][action]:
                    # calculate the expected value
                    v += action_prob * state_prob * (reward + discount * V[next_state])
            # claculate the absolute change of value function
            delta = max(delta, np.abs(V[state] - v))
            # update value function
            V[state] = v
        eval_iters += 1
        
        # terminate if value change is insignificant
        if delta < theta:
            print(f'Policy evaluation terminated after {eval_iters} iterations.\n')
            return V

### **Create a Policy Iteration function with the following parameters**
* policy: 2D array of a size n(S) x n(A), each cell represents a probability of taking action a in state s.
* environment: Initialized OpenAI gym environment object
* discount_factor: MDP discount factor.
* theta:  A  threshold  of  a  value  function  change.  Once  the  update  to  value function is below this number
* max_iterations: Maximum number of iterations 

In [5]:
def policy_iteration(env, discount=1.0, max_iter=1000):

    n_states = env.observation_space.n
    n_actions = env.action_space.n
    
    # start with random policy = n_states x n_actions / n_actions
    policy = np.ones(shape=[n_states, n_actions]) / n_actions

    # init counter of evaluated policies
    evaluated_policies = 1
    
    # repeat until convergence or critical number of iterations reached
    for i in range(int(max_iter)):
        stable_policy = False
        # Evaluate current policy
        V = policy_eval(policy, env, discount)
        # go through each state & try to improve actions that were taken
        for state in range(n_states):
            curr_action = np.argmax(policy[state])
            # look one step ahead and evaluate if curr_action is optimal
            # will try every possible action in a curr_state
            action_value = one_step_lookahead(env, state, V, discount)
            # select best aciton 
            best_action = np.argmax(action_value)
            # if action didn't change
            if curr_action != best_action:
                stable_policy = True
            # Greedy policy update
            policy[state] = np.eye(n_actions)[best_action]
        evaluated_policies += 1
        # if the algo converged & policy is not changing anymore
        if stable_policy:
            print(f'Found stable policy after {evaluated_policies:,} evaluations.\n')
            return policy, V

### **Create a Value Iteration function with the following parameters**
* environment: Initialized OpenAI gym environment object
* discount_factor: MDP discount factor
* theta:  A  threshold  of  a  value  function  change.  Once  the  update  to  value function is below this numberd.
* max_iterations: Maximum number of iterations 

In [6]:
# defining Value iteration algorithm to solve MDP.

def value_iteration(env, discount=1e-1, theta=1e-9, max_iter=1e4):

    # initalized state-value function with zeros for each env state
    V = np.zeros(env.observation_space.n)
    
    for i in range(int(max_iter)):
        # early stopping condition
        delta = 0
        # update each state
        for state in range(env.observation_space.n):
            # Do a one-step lookahead to calculate state-action values
            action_value = one_step_lookahead(env, state, V, discount)
            # select best action to perform based on the highest state-action values
            best_action_value = np.max(action_value)
            # calculate change in value
            delta = max(delta, np.abs(V[state] - best_action_value))
            # update the value function for current state
            V[state] = best_action_value
            
        # check if we can stop
        if delta < theta:
            print(f'\nValue iteration converged at iteration #{i+1:,}')
            break
    
    # create deterministic policy using the optimal value function
    policy = np.zeros(shape=[env.observation_space.n, env.action_space.n])
    
    for state in range(env.observation_space.n):
        # one step lookahead to find the best action for this state
        action_value = one_step_lookahead(env, state, V, discount)
        #select the best action based on the highest state-action value
        best_action = np.argmax(action_value)
        # update the policy to perform a better action at a current state
        policy[state, best_action] = 1.0
    
    return policy, V

### Defining a function to implement eacdh episode

In [12]:
def play_episodes(env, episodes, policy, max_action=100, render=False):

    wins = 0
    total_reward, total_action = 0, 0
    
    for episode in range(episodes):
        state = env.reset()
        done, max_a = False, 0
        while max_a < max_action:
            action = np.argmax(policy[state])
            next_state, reward, done, _ = env.step(action)
            if render:
                env.render()
            max_a += 1  # increment actions taken
            total_reward += reward  # increment reward received
            state = next_state  # set current state to next state
            # terminate if we're done and increment `wins`
            if done:
                wins += 1
                break
        
        total_action += max_a

    print(f'Total rewards: {total_reward:,}\tMax action: {max_a:,}')
    
    avg_reward = total_reward / episodes
    avg_action = total_action / episodes
    print('')
    '''
    return: tuple(wins, total_reward, average_reward)
        - wins: The total number of wins the agent has
        - total_reward: The agent's total accumulated reward
        - average_reward: The agent's average reward
    '''
    return wins, total_reward, avg_reward, avg_action

### Implementing the **agent** in the given environment for 1000 episodes

In [23]:
episodes = 1000

def agent(env):

    rewards = []
  
    action_mapping = {
        0: '\u2191',  # up
        1: '\u2192',  # right
        2: '\u2193',  # down
        3: '\u2190'   # left
    }
    
    policies = [
        ('Policy Iteration', policy_iteration),
        ('Value Iteration', value_iteration)
    ]
    
    for iter_name, iter_func in policies:
        policy, V = iter_func(env)
        
        print(f'Final policy using {iter_name}:')
        print(' '.join([action_mapping[action] for action in np.argmax(policy, axis=1)]))
        
        wins, total_reward, avg_reward, avg_action = play_episodes(env, episodes, policy)
        rewards.append(total_reward)
        
        print(f'number of wins = {wins:,}')
        print(f'average reward = {avg_reward:.2f}')
        print(f'average action = {avg_action:.2f}')

    return rewards




In [24]:
rewards = agent(env)

Policy evaluation terminated after 66 iterations.

Found stable policy after 2 evaluations.

Final policy using Policy Iteration:
↑ ← ↑ ← ↑ ↑ ↑ ↑ ← → ↑ ↑ ↑ ↓ → ↑
Total rewards: 720.0	Max action: 61

number of wins = 1,000
average reward = 0.72
average action = 41.19

Value iteration converged at iteration #8
Final policy using Value Iteration:
→ ← ↓ ← ↑ ↑ ↑ ↑ ← → ↑ ↑ ↑ ↓ → ↑
Total rewards: 443.0	Max action: 14

number of wins = 1,000
average reward = 0.44
average action = 28.35


In [25]:
#env.close()

In [26]:
print(rewards)

[720.0, 443.0]
